In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

from processor import NLProcessor

from tqdm import tnrange

In [ ]:
corrupt_rate           = 0.1

In [ ]:
processor = NLProcessor()
processor.load_spam()

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=1, solver = "lbfgs",
                           max_iter = 800,
                           fit_intercept= False,
                           warm_start = True)

def evaluate(model):
    predictions = model.predict(processor.X_test)
    return accuracy_score(processor.Y_test, predictions)
    
    
accuracy_scores = []
guess_scores  = []
for seed in range(15):

    X_train,  ycrptd = processor.corrupt_random(seed, corrupt_rate)
    model.fit(X_train, ycrptd)
    
    # Let us calculate the initial score of the model
    initial_score = evaluate(model)
    accuracy_scores.append([initial_score])
    guess_scores.append([0])
    
    # Let us make some rounds of correction of sample based on training loss
    correction_rounds = int(2 * X_train.shape[0] * corrupt_rate)
    
    # Get influences
    influences = processor.training_influence(model, ycrptd, C = 1)
    order = np.argsort(influences)
    
    for i in tnrange(correction_rounds):
        to_check = order[i]
        if ycrptd[to_check] == processor.Y_train[to_check]:
            # The guess was incorect. No need to retrain the model
            score = accuracy_scores[-1][-1]
            correct_guess = 0
        else:
            # It was a good guess
            ycrptd[to_check] = processor.Y_train[to_check]
            model.fit(X_train, ycrptd)
            score = evaluate(model)
            correct_guess = 1
        
        accuracy_scores[-1].append(score)
        guess_scores[-1].append(guess_scores[-1][-1] + correct_guess)

In [ ]:
accuracy_array = np.array(accuracy_scores)
guess_array  = np.array(guess_scores)

accuracy_means = np.mean(accuracy_array[:, ::15], axis = 0)
accuracy_mins  = np.min(accuracy_array[:, ::15], axis  = 0)
accuracy_maxs  = np.max(accuracy_array[:, ::15], axis  = 0)
stacked      = np.stack([accuracy_means -accuracy_mins, accuracy_maxs- accuracy_means])

plt.figure(figsize=(8,8))
plt.errorbar(range(len(accuracy_array[0]))[::15], accuracy_means, yerr = stacked)
plt.show()

In [ ]:
guess_means = np.mean(guess_array[:, ::15]/(X_train.shape[0] * corrupt_rate), axis = 0)
guess_mins  = np.min(guess_array[:, ::15]/(X_train.shape[0] * corrupt_rate), axis  = 0)
guess_maxs  = np.max(guess_array[:, ::15]/(X_train.shape[0] * corrupt_rate), axis  = 0)
stacked      = np.stack([guess_means -guess_mins, guess_maxs- guess_means])

plt.figure(figsize=(8,8))
plt.errorbar(range(len(guess_array[0]))[::15], guess_means, yerr = stacked, label = 'Influence Correction')
plt.plot(range(len(guess_array[0]))[:int(X_train.shape[0] * corrupt_rate):15], 
         np.linspace(0, 1, int(X_train.shape[0] * corrupt_rate))[::15],  label = 'Oracle')
plt.legend()
plt.show()

In [ ]:
guess_means_inf = np.mean(guess_array/(X_train.shape[0] * corrupt_rate), axis = 0)

In [ ]:
%store guess_means_inf